In [11]:
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.widgets import MultiCursor, Slider
import matplotlib.figure as figs

import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

import uncertainties as uc
from uncertainties import ufloat as uf
from uncertainties import unumpy as unp
from timescan_plot import timescanplot
from FELion_definitions import ShowInfo, ErrorInfo
%matplotlib tk

In [13]:
files1 = ['16_11_18-1.scan', '16_11_18-2.scan']
files2 = ['16_11_18-4.scan', '16_11_18-3.scan']
files3 = ['16_11_18-1.scan', '16_11_18-2.scan', '16_11_18-4.scan', '16_11_18-3.scan']
location = r'D:\FELion-Spectrum-Analyser\testing\_datas'

power1, power2, power3 = [12, 14.5], [22, 25], [12, 14.5, 22, 25]
depletionPlot1(files1, location, save=False, show=True, power_values=power1, n=10, close = False)

{'res': 'm03_ao13_reso', 'b0': 'm03_ao09_width'}
{'res': 2.0, 'b0': 100000.0}
{'res': 'm03_ao13_reso', 'b0': 'm03_ao09_width'}
{'res': 2.0, 'b0': 100000.0}


In [38]:
temp = '12, 14.5, 10'
temp = np.asarray(temp.split(','), dtype=np.float)
temp[:2]

array([12. , 14.5])

In [2]:
def depletionPlot1(files, location, save, show, power_values, n, close=False):
    
    if len(files)>2: return ShowInfo('Info', 'Please select only 2-files')
    power_values = np.array(power_values)
    np.seterr(all='ignore')
    os.chdir(location)
    
    fig0, axs0 = plt.subplots()
    
    lg_fontsize = 15
    title_fontsize = 15
    lb_size = 15
    
    counts, stde = [], []
    for f in files:        
        mass, iterations, t_res, t_b0, mean, error, time = timescanplot(f, location, save, show, depletion=True)
        axs0.errorbar(time, mean[0], yerr = error[0], label = '{}; {}:[{}], B0:{}ms, Res:{}'.format(f, mass[0], iterations[0], t_b0, t_res))
        
        time = time[1:]/1000
        mean = mean[0][1:]
        stde.append(error[0][1:])
        counts.append(mean)
        
    counts, stde = np.array(counts), np.array(stde)
    
    axs0.set_title('Timescan', fontsize=title_fontsize)
    axs0.set_xlabel('time (s)', fontsize= lb_size)
    axs0.set_ylabel('Counts', fontsize= lb_size)
    axs0.grid()
    axs0.legend()
    
    on_off = []
    for i in counts:
        on_off.append(i.min())
    on_off = np.array(on_off)
    
    K_OFF, N = [], []
    K_OFF_err, N_err = [], []
    
    K_ON, Na0, Nn0 = [], [], []
    K_ON_err, Na0_err, Nn0_err = [], [], []
    
    fig, axs = plt.subplots(figsize=(25, 10), dpi=70)
    
    plt.subplots_adjust(
        top = 0.95,
        bottom = 0.2,
        left = 0.05,
    )
    
    for i in range(0, len(counts), 2):
        
        on = np.argmin(on_off)
        off = np.argmax(on_off)
        
        # making the error '0' value as very close to '0' 
        #since div by it makes it easier while fitting parameters
        stde[on][stde[on]==0]=10e-10
        stde[off][stde[off]==0]=10e-10
        
        #depletion values; y-axis
        depletion_on, depletion_on_err = counts[on], stde[on]
        depletion_off, depletion_off_err = counts[off], stde[off]

        # power values; x-axis
        power_on = (power_values[on]*n*time)/1000. # divide by 1000 for mJ to J conversion 
        power_off = (power_values[off]*n*time)/1000.
        power_max = power_values.max()*n*time.max()/1000.
        x = np.linspace(0, power_max, num=len(time))
        
        axs.errorbar(power_off, depletion_off, yerr = depletion_off_err, fmt='ok')
        axs.errorbar(power_on, depletion_on, yerr = depletion_on_err, fmt='ok')
        
        ### finding parameters for fitting
        
        # depletion off
        def N_OFF(x, K_OFF, N):
            return (N)*np.exp(-K_OFF*x)
        
        K_OFF_init, N_init = 0, depletion_off.max()
        
        N_increase_bound_by = 1000
        N_upper_bound = N_init + N_increase_bound_by
        
        pop_off, popc_off = curve_fit(
            N_OFF, power_off, depletion_off,
            sigma = stde[off],
            absolute_sigma = True,
            p0 = [K_OFF_init, N_init],
            bounds = [(-np.inf, 0), (np.inf, N_upper_bound)]
            )
        
        perr_off = np.sqrt(np.diag(popc_off))
        
        # off fitting variables         
        K_OFF.append(pop_off[0])
        N.append(pop_off[1])
        
        K_OFF_err.append(perr_off[0])
        N_err.append(perr_off[1])

        # depletion on
        def N_ON(X, Na0, Nn0, K_ON):
            x, K_OFF = X
            return Na0*np.exp(-K_ON*x)*np.exp(-K_OFF*x) + Nn0*np.exp(-K_OFF*x)

        #K_ON_init, Na0_init, Nn0_init = ()
        X = (power_on, pop_off[0])
        pop_on, popc_on = curve_fit(
            N_ON, X, depletion_on,
            sigma = stde[on],
            absolute_sigma = True,
            #p0 = [Na0_init, Nn0_init, K_ON_init]
            bounds = ([0,0,-np.inf], [pop_off[1], pop_off[1], np.inf])
        )
        perr_on = np.sqrt(np.diag(popc_on))
        
        #on fitting variables
        Na0.append(pop_on[0])
        Nn0.append(pop_on[1])
        K_ON.append(pop_on[2])
        
        Na0_err.append(perr_on[0])
        Nn0_err.append(perr_on[1])
        K_ON_err.append(perr_on[2])
    
    uK_OFF, uN = unp.uarray(K_OFF, K_OFF_err), unp.uarray(N, N_err)
    uK_ON, uNa0, uNn0 = unp.uarray(K_ON, K_ON_err), unp.uarray(Na0, Na0_err) , unp.uarray(Nn0, Nn0_err)
    
    ## depletion plot
    box0 = axs.get_position() ##[left, bottom, width, height]
    depletion_plot_position = [box0.x0+0.55, box0.y0-0.1, box0.width*0.45, box0.height*0.9]
    depletion_plot = plt.axes(depletion_plot_position)
    
    def Depletion(X, A):
        x, K_ON = X
        return A*(1-np.exp(-K_ON*x))
    
    uy_OFF = lambda x, uN, uK_OFF: uN*unp.exp(-uK_OFF*x)
    uy_ON = lambda x, uNa0, uNn0, uK_OFF, uK_ON : uNa0*unp.exp(-uK_ON*x)*unp.exp(-uK_OFF*x) + uNn0*unp.exp(-uK_OFF*x)
    
    A, A_err = [], []
    
    for i in range(len(N)):
        
        udepletion = 1 - uy_ON(x, uNa0[i], uNn0[i], uK_OFF[i], uK_ON[i])/uy_OFF(x, uN[i], uK_OFF[i])
        depletion, depletion_error = unp.nominal_values(udepletion), unp.std_devs(udepletion)
        
        #fitting for depletion
        X = (x, K_ON[i])
        pop_depletion, poc_depletion = curve_fit(
            Depletion, X, depletion, 
            sigma = depletion_error,
            absolute_sigma = True
        )
        
        A.append(pop_depletion[0])
        perr_A = np.sqrt(np.diag(poc_depletion))
        A_err.append(perr_A[0])

    uA = unp.uarray(A, A_err)

    def plot(i, l):
        # off plotting
        y_off0 = N_OFF(x, K_OFF[i], N[i])
        g_off0, = axs.plot(x, y_off0, label = 'N_OFF: [{:.2f}mJ], K_OFF={:.2fP}/J, N={:.2fP}'.format(power_values[i+1], uK_OFF[i], uN[i]))

        # on plotting
        y_on0 = N_ON((x, K_OFF[i]), Na0[i], Nn0[i], K_ON[i])
        g_on0, = axs.plot(x, y_on0, label = 'N_ON: [{:.2f}mJ], K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(power_values[i], uK_ON[i], uNa0[i]+uNn0[i], uNa0[i], uNn0[i]))
        
        # deletion plot
        udepletion_new = 1 - uy_ON(x, uNa0[i], uNn0[i], uK_OFF[i], uK_ON[i])/uy_OFF(x, uN[i], uK_OFF[i])
        depletion_new, depletion_error_new = unp.nominal_values(udepletion_new), unp.std_devs(udepletion_new)

        depletion0, = depletion_plot.plot(x, depletion_new, '.k')
        
        depletion_fitted = Depletion(X, A[i])
        depletion1, = depletion_plot.plot(x, depletion_fitted,
                                         label = 'A = {:.2fP}, K_ON = {:.2fP}/J'.format(uA[i], uK_ON[i])
                                         )
        
        # controlling fitting parameters
        axcolor = 'lightgoldenrodyellow'
        
        koff_g = plt.axes([l, 0.12, 0.2, 0.015], facecolor=axcolor) #[left, bottom, width, height]
        n_g = plt.axes([l, 0.10, 0.2, 0.015], facecolor=axcolor)

        kon_g = plt.axes([l, 0.08, 0.2, 0.015], facecolor=axcolor)
        na_g = plt.axes([l, 0.06, 0.2, 0.015], facecolor=axcolor)
        nn_g = plt.axes([l, 0.04, 0.2, 0.015], facecolor=axcolor)

        koff_slider = Slider(koff_g, '$K_{OFF}$', 0, K_OFF[i]+10, valinit = K_OFF[i])
        n_slider = Slider(n_g, 'N', 0, N[i]+(N[i]/2), valinit = N[i])

        kon_slider = Slider(kon_g, '$K_{ON}$', 0, K_ON[i]+10, valinit = K_ON[i])
        na_slider = Slider(na_g, '$Na_0$', 0, Na0[i]+(Na0[i]/2), valinit = Na0[i])
        nn_slider = Slider(nn_g, '$Nn_0$', 0, Nn0[i]+(Nn0[i]/2), valinit = Nn0[i])
                
        def update(val):
            
            koff = koff_slider.val
            ukoff = uf(koff, K_OFF_err[i])

            n = n_slider.val
            un = uf(n, N_err[i])

            kon = kon_slider.val
            ukon = uf(kon, K_ON_err[i])

            na = na_slider.val
            una = uf(na, Na0_err[i])

            nn = nn_slider.val
            unn = uf(nn, Nn0_err[i])

            yoff = N_OFF(x, koff, n)
            g_off0.set_ydata(yoff)

            yon = N_ON((x, koff), na, nn, kon)
            g_on0.set_ydata(yon)
             
            # depletion
            udepletion_new1 = 1 - uy_ON(x, una, unn, ukoff, ukon)/uy_OFF(x, un, ukoff)
            depletion_new1, depletion_error_new1 = unp.nominal_values(udepletion_new1), unp.std_devs(udepletion_new1)
            depletion0.set_ydata(depletion_new1)
              
            X = (x, kon)
            pop_depletion, poc_depletion = curve_fit(
                    Depletion, X , depletion_new1, 
                    sigma = depletion_error_new1, 
                    absolute_sigma = True
                )
            
            A_new1 = pop_depletion[0]
            perr = np.sqrt(np.diag(poc_depletion))[0]
            uA_new1 = uf(A_new1 , perr)
    
            depletion_fitted_new = Depletion(X, A_new1)
            depletion1.set_ydata(depletion_fitted_new)

            k = i*2
            legend.get_texts()[k].set_text('N_OFF: [{:.2f}mJ], K_OFF={:.2fP}/J, N={:.2fP}'.format(power_values[i+1], ukoff, un))
            legend.get_texts()[k+1].set_text('N_ON: [{:.2f}mJ], K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(power_values[i], ukon, una+unn, una, unn))
            depletion_legend.get_texts()[i].set_text('A = {:.2fP}, K_ON = {:.2fP}/J'.format(uA_new1, ukon))

            fig.canvas.draw_idle()
            
            return fig

        koff_slider.on_changed(update)
        n_slider.on_changed(update)

        kon_slider.on_changed(update)
        na_slider.on_changed(update)
        nn_slider.on_changed(update)
    
        return koff_slider, n_slider, kon_slider, na_slider, nn_slider, koff_g, n_g, kon_g, na_g, nn_g, fig

    widget_position = l = 0.05
    for i in range(len(N)):
        koff_slider, n_slider, kon_slider, na_slider, nn_slider, koff_g, n_g, kon_g, na_g, nn_g, fig = plot(i, l)
        l += 0.25

    ### setting labels
    title_depletion1 = '$N_{ON}(ntE)=N_{a0}e^{-k_{on}ntE}e^{-k_{off}ntE} + N_{n0}e^{-k_{off}ntE}$ ;\t$N_{OFF}(ntE)=(N)e^{-k_{off}ntE}$ ; $N = N_{a0}+ N_{n0}$'
    axs.set_title(title_depletion1, fontsize=title_fontsize)
    axs.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    axs.set_ylabel('Counts', fontsize= lb_size)

    axs.grid(True)
    box = axs.get_position()
    axs.set_position([box.x0, box.y0, box.width*0.6, box.height])
    legend = axs.legend(loc='center left', bbox_to_anchor=(1, 0.95), title=files, fontsize=lg_fontsize-2)
    legend.get_title().set_fontsize(lg_fontsize)
    
    depletion_plot.grid(True)
    depletion_legend = depletion_plot.legend(loc = 'lower right', fontsize=lg_fontsize)
    depletion_plot.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    depletion_plot.set_ylabel('Relative abundance of active isomer', fontsize= lb_size)
    depletion_plot.set_title('$D(ntE) = 1-N_{ON}/N_{OFF}$ fitted with $D(ntE) = A(1-e^{K_{ON}*ntE})$', fontsize = title_fontsize)
    
    if save: plt.savefig("Depletion.pdf", bbox_inches='tight')
    if show: plt.show()
    if close: plt.close('all')

In [59]:
def depletionPlot2(files, location, save, show, power_values, n, close=False):
    
    power_values = np.array(power_values)
    np.seterr(all='ignore')
    os.chdir(location)
    
    fig, axs = plt.subplots(figsize=(25, 10), dpi=70)
    plt.subplots_adjust(
        top = 0.95,
        bottom = 0.2,
        left = 0.05,
    )
    
    lg_fontsize = 15
    title_fontsize = 15
    lb_size = 15
    
    counts, stde = [], []
    for f in files:        
        mass, iterations, mean, error, time = timescanplot(f, location, save, show, depletion=True)
        time = time[1:]/1000
        mean = mean[0][1:]
        stde.append(error[0][1:])
        counts.append(mean)
        
    counts, stde = np.array(counts), np.array(stde)
    
    K_OFF, N = [], []
    K_OFF_err, N_err = [], []
    
    K_ON, Na0, Nn0 = [], [], []
    K_ON_err, Na0_err, Nn0_err = [], [], []
    
    for i in range(0, len(counts), 2):
        on = i
        off = i+1
        
        # making the error '0' value as very close to '0' 
        #since div by it makes it easier while fitting parameters
        stde[on][stde[on]==0]=10e-10
        stde[off][stde[off]==0]=10e-10
        
        #depletion values; y-axis
        depletion_on, depletion_on_err = counts[on], stde[on]
        depletion_off, depletion_off_err = counts[off], stde[off]

        # power values; x-axis
        power_on = (power_values[on]*n*time)/1000. # divide by 1000 for mJ to J conversion 
        power_off = (power_values[off]*n*time)/1000.
        power_max = power_values.max()*n*time.max()/1000.
        x = np.linspace(0, power_max, num=len(time))
        
        axs.errorbar(power_off, depletion_off, yerr = depletion_off_err, fmt='ok')
        axs.errorbar(power_on, depletion_on, yerr = depletion_on_err, fmt='ok')
        
        ### finding parameters for fitting
        
        # depletion off
        def N_OFF(x, K_OFF, N):
            return (N)*np.exp(-K_OFF*x)
        
        K_OFF_init, N_init = 0, depletion_off.max()
        
        N_increase_bound_by = 1000
        N_upper_bound = N_init + N_increase_bound_by
        
        pop_off, popc_off = curve_fit(
            N_OFF, power_off, depletion_off,
            sigma = stde[off],
            absolute_sigma = True,
            p0 = [K_OFF_init, N_init],
            bounds = [(-np.inf, 0), (np.inf, N_upper_bound)]
            )
        
        perr_off = np.sqrt(np.diag(popc_off))
        
        # off fitting variables         
        K_OFF.append(pop_off[0])
        N.append(pop_off[1])
        
        K_OFF_err.append(perr_off[0])
        N_err.append(perr_off[1])

        # depletion on
        def N_ON(X, Na0, Nn0, K_ON):
            x, K_OFF = X
            return Na0*np.exp(-K_ON*x)*np.exp(-K_OFF*x) + Nn0*np.exp(-K_OFF*x)

        #K_ON_init, Na0_init, Nn0_init = ()
        X = (power_on, pop_off[0])
        pop_on, popc_on = curve_fit(
            N_ON, X, depletion_on,
            sigma = stde[on],
            absolute_sigma = True,
            #p0 = [Na0_init, Nn0_init, K_ON_init]
            bounds = ([0,0,-np.inf], [pop_off[1], pop_off[1], np.inf])
        )
        perr_on = np.sqrt(np.diag(popc_on))
        
        #on fitting variables
        Na0.append(pop_on[0])
        Nn0.append(pop_on[1])
        K_ON.append(pop_on[2])
        
        Na0_err.append(perr_on[0])
        Nn0_err.append(perr_on[1])
        K_ON_err.append(perr_on[2])
    
    uK_OFF, uN = unp.uarray(K_OFF, K_OFF_err), unp.uarray(N, N_err)
    uK_ON, uNa0, uNn0 = unp.uarray(K_ON, K_ON_err), unp.uarray(Na0, Na0_err) , unp.uarray(Nn0, Nn0_err)
    
    ## depletion plot
    box0 = axs.get_position() ##[left, bottom, width, height]
    depletion_plot_position = [box0.x0+0.55, box0.y0-0.1, box0.width*0.45, box0.height*0.9]
    depletion_plot = plt.axes(depletion_plot_position)
    
    def Depletion(X, A):
        x, K_ON = X
        return A*(1-np.exp(-K_ON*x))
    
    uy_OFF = lambda x, uN, uK_OFF: uN*unp.exp(-uK_OFF*x)
    uy_ON = lambda x, uNa0, uNn0, uK_OFF, uK_ON : uNa0*unp.exp(-uK_ON*x)*unp.exp(-uK_OFF*x) + uNn0*unp.exp(-uK_OFF*x)
    
    A, A_err = [], []
    
    for i in range(len(N)):
        
        udepletion = 1 - uy_ON(x, uNa0[i], uNn0[i], uK_OFF[i], uK_ON[i])/uy_OFF(x, uN[i], uK_OFF[i])
        depletion, depletion_error = unp.nominal_values(udepletion), unp.std_devs(udepletion)
        
        #fitting for depletion
        X = (x, K_ON[i])
        pop_depletion, poc_depletion = curve_fit(
            Depletion, X, depletion, 
            sigma = depletion_error,
            absolute_sigma = True
        )
        
        A.append(pop_depletion[0])
        perr_A = np.sqrt(np.diag(poc_depletion))
        A_err.append(perr_A[0])

    uA = unp.uarray(A, A_err)

    class plot:
        
        def __init__(self, i, l, fig):
            self.i = i
            self.l = l
            self.fig = fig
        
        def plotting(self):

            ## off plotting
            self.y_off0 = N_OFF(x, K_OFF[self.i], N[self.i])
            self.g_off0, = axs.plot(x, self.y_off0, label = 'N_OFF: [{:.2f}mJ], K_OFF={:.2fP}/J, N={:.2fP}'.format(power_values[self.i+1], uK_OFF[self.i], uN[self.i]))

            # on plotting
            self.y_on0 = N_ON((x, K_OFF[self.i]), Na0[self.i], Nn0[self.i], K_ON[self.i])
            self.g_on0, = axs.plot(x, self.y_on0, label = 'N_ON: [{:.2f}mJ], K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(power_values[self.i], uK_ON[self.i], uNa0[self.i]+uNn0[self.i], uNa0[self.i], uNn0[self.i]))

            # deletion plot
            self.udepletion_new = 1 - uy_ON(x, uNa0[self.i], uNn0[self.i], uK_OFF[self.i], uK_ON[self.i])/uy_OFF(x, uN[self.i], uK_OFF[self.i])
            self.depletion_new, self.depletion_error_new = unp.nominal_values(self.udepletion_new), unp.std_devs(self.udepletion_new)

            self.depletion0, = depletion_plot.plot(x, self.depletion_new, '--')

            self.X = (x, K_ON[self.i])
            self.depletion_fitted = Depletion(X, A[self.i])
            self.depletion1, = depletion_plot.plot(x, self.depletion_fitted,
                                             label = 'A = {:.2fP}, K_ON = {:.2fP}/J'.format(uA[self.i], uK_ON[self.i])
                                             )

            # controlling fitting parameters
            axcolor = 'lightgoldenrodyellow'

            self.koff_g = plt.axes([self.l, 0.12, 0.2, 0.015], facecolor=axcolor) #[left, bottom, width, height]
            self.n_g = plt.axes([self.l, 0.10, 0.2, 0.015], facecolor=axcolor)

            self.kon_g = plt.axes([self.l, 0.08, 0.2, 0.015], facecolor=axcolor)
            self.na_g = plt.axes([self.l, 0.06, 0.2, 0.015], facecolor=axcolor)
            self.nn_g = plt.axes([self.l, 0.04, 0.2, 0.015], facecolor=axcolor)

            self.koff_slider = Slider(self.koff_g, '$K_{OFF}$', 0, K_OFF[self.i]+10, valinit = K_OFF[self.i])
            self.n_slider = Slider(self.n_g, 'N', 0, N[self.i]+(N[self.i]/2), valinit = N[self.i])

            self.kon_slider = Slider(self.kon_g, '$K_{ON}$', 0, K_ON[self.i]+10, valinit = K_ON[self.i])
            self.na_slider = Slider(self.na_g, '$Na_0$', 0, Na0[self.i]+(Na0[self.i]/2), valinit = Na0[self.i])
            self.nn_slider = Slider(self.nn_g, '$Nn_0$', 0, Nn0[self.i]+(Nn0[self.i]/2), valinit = Nn0[self.i])
            
            self.koff_slider.on_changed(self.update)
            self.n_slider.on_changed(self.update)

            self.kon_slider.on_changed(self.update)
            self.na_slider.on_changed(self.update)
            self.nn_slider.on_changed(self.update)

        def update(self, val):

            self.koff = self.koff_slider.val
            self.ukoff = uf(self.koff, K_OFF_err[self.i])

            self.n = self.n_slider.val
            self.un = uf(self.n, N_err[self.i])

            self.kon = self.kon_slider.val
            self.ukon = uf(self.kon, K_ON_err[self.i])

            self.na = self.na_slider.val
            self.una = uf(self.na, Na0_err[self.i])

            self.nn = self.nn_slider.val
            self.unn = uf(self.nn, Nn0_err[self.i])

            self.yoff = N_OFF(x, self.koff, self.n)
            self.g_off0.set_ydata(self.yoff)

            self.yon = N_ON((x, self.koff), self.na, self.nn, self.kon)
            self.g_on0.set_ydata(self.yon)

            # depletion
            self.udepletion_new1 = 1 - uy_ON(x, self.una, self.unn, self.ukoff, self.ukon)/uy_OFF(x, self.un, self.ukoff)
            self.depletion_new1, self.depletion_error_new1 = unp.nominal_values(self.udepletion_new1), unp.std_devs(self.udepletion_new1)
            self.depletion0.set_ydata(self.depletion_new1)

            self.X = (x, self.kon)
            self.pop_depletion, self.poc_depletion = curve_fit(
                    Depletion, self.X , self.depletion_new1, 
                    sigma = self.depletion_error_new1, 
                    absolute_sigma = True
                )

            self.A_new1 = self.pop_depletion[0]
            self.perr = np.sqrt(np.diag(self.poc_depletion))[0]
            self.uA_new1 = uf(self.A_new1 , self.perr)

            self.depletion_fitted_new = Depletion(self.X, self.A_new1)
            self.depletion1.set_ydata(self.depletion_fitted_new)

            self.k = self.i*2
            legend.get_texts()[self.k].set_text('N_OFF: [{:.2f}mJ], K_OFF={:.2fP}/J, N={:.2fP}'.format(power_values[self.i+1], self.ukoff, self.un))
            legend.get_texts()[self.k+1].set_text('N_ON: [{:.2f}mJ], K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(power_values[self.i], self.ukon, self.una+self.unn, self.una, self.unn))
            depletion_legend.get_texts()[self.i].set_text('A = {:.2fP}, K_ON = {:.2fP}/J'.format(self.uA_new1, self.ukon))

            self.fig.canvas.draw_idle()
            
            
    def on_key(event):
        for i in range(len(N)):
            if event.key == str(i):
                l = 0.05+(0.25)**i
                temp = plot(i, l, fig)
                temp.plotting()
                
    cid = fig.canvas.mpl_connect('key_press_event', on_key)
    
    
    ### setting labels
    title_depletion1 = '$N_{ON}(ntE)=N_{a0}e^{-k_{on}ntE}e^{-k_{off}ntE} + N_{n0}e^{-k_{off}ntE}$ ;\t$N_{OFF}(ntE)=(N)e^{-k_{off}ntE}$ ; $N = N_{a0}+ N_{n0}$'
    axs.set_title(title_depletion1, fontsize=title_fontsize)
    axs.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    axs.set_ylabel('Counts', fontsize= lb_size)

    axs.grid(True)
    box = axs.get_position()
    axs.set_position([box.x0, box.y0, box.width*0.6, box.height])
    legend = axs.legend(loc='center left', bbox_to_anchor=(1, 0.95), title=files, fontsize=lg_fontsize-2)
    legend.get_title().set_fontsize(lg_fontsize)
    
    depletion_plot.grid(True)
    depletion_legend = depletion_plot.legend(loc = 'lower right', fontsize=lg_fontsize)
    depletion_plot.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    depletion_plot.set_ylabel('Relative abundance of active isomer', fontsize= lb_size)
    depletion_plot.set_title('$D(ntE) = 1-N_{ON}/N_{OFF}$ fitted with $D(ntE) = A(1-e^{K_{ON}*ntE})$', fontsize = title_fontsize)
    
    if save: plt.savefig("Depletion.pdf", bbox_inches='tight')
    if show: plt.show()
    if close: plt.close('all')

In [60]:

files1 = ['16_11_18-1.scan', '16_11_18-2.scan']
files2 = ['16_11_18-4.scan', '16_11_18-3.scan']
files3 = ['16_11_18-1.scan', '16_11_18-2.scan', '16_11_18-4.scan', '16_11_18-3.scan']
location = r'D:\FELion-Spectrum-Analyser\testing\_datas'
%matplotlib tk
power1, power2, power3 = [12, 14.5], [22, 25], [12, 14.5, 22, 25]
depletionPlot2(files3, location, save=False, show=True, power_values=power3, n=10, close = False)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "
